股票系統介紹
===
股票測試網址: https://merry.ee.ncku.edu.tw/stock/  
帳號為luffy帳號或學號，密碼為成功入口密碼

## 介紹
伺服器上有準備從2004年到現在每天台股的股市資料，透過這些原始資料，要實作一個預測器來決定使用者明天要怎麼投資股票，購買和賣出的結果可以由上述的網址上登入後看到詳細行為，而每個人的目前的總資產會依照時間化成圖和其他人比較。
## 股票資料
在luffy的 `/home/mlb/res/stock/twse/json/` 資料夾中，存放著每一天每一支股票的編號、開牌價、收盤價、最高最低價、成交量等資料。  
資料的格式如下範例:

In [ ]:
stock =   {
             "0050" : {
                "adj_close" : "84.90",
                "close" : "84.95",
                "high" : "85.80",
                "low" : "84.90",
                "open" : "85.80",
                "volume" : "20996200"
             },
            "0051" : {
               "adj_close" : "32.50",
               "close" : "32.50",
               "high" : "33.20",
               "low" : "32.46",
               "open" : "33.20",
               "volume" : "92075"
            },
            "0052" : {
               "adj_close" : "54.30",
               "close" : "54.50",
               "high" : "54.90",
               "low" : "54.50",
               "open" : "54.90",
               "volume" : "102150"
            }
    }

print(stock['0050'])
print(stock['0050']['close'])

## 產生預測模型
為了訓練股票預測器，必須從上述的json股票資料中根據 **自定義** 的時間區段、股票特徵種類等切分出訓練資料。
例如: 取預測日當日前一周的開盤價、收盤價以及平均的價格來訓練模型。

## 產生Decision File
Decision File指的是最終決定要買哪一隻股票、要買多少錢以及要讓這支股票存活多久等條件。Decision File的檔名格式為`YYYY-MM-DD.json`，日期是預測日當日。而Decision File的資料格式也是json，是以一個array包在最外層，每一支股票用一個object表示。
  
通常會根據上述的預測結果來決定Decision File要買(放空)哪支股票，但是單一預測結果通常只能告訴你某支股票會漲、會跌或是他預測股價是多少，除非建立多個模型來決定每一個Decision File中的參數，否則像是權重、存放時間等等的參數都需要使用者另外寫條件來判斷。

Decision File格式範例如下:

In [ ]:
[ # 一個陣列，每個元素是一個 object
  {
    "code": "1214", # 股票代碼
    "type": "buy", # 兩種操作，買(buy)/放空(short)
    "weight": 1, # 買幾張股票; 每個人一開始有 1 單位的虛擬貨幣(以這個例子為例，用 1/4 買 1214；用 3/4 放空 2351)
    "life": 5, # 5 天後會賣(其中一個條件)
    "open_price": 16.75,# 等於或小於這個價格買
    "close_high_price": 16.8,# 比這個價格高就賣
    "close_low_price": 16.3 # 停損點
  },
  {
    "code": "2351",
    "life": 3,
    "type": "short",
    "weigth": 3,
    "open_price": 27.3,
    "close_high_price": 27.5,
    "close_low_price": 25.8
  }
]


## 股票權重
每個人每一天有一單位的錢可以花，在計算上系統會將所有權重加起來算成百分比。假設只買兩支股票，1214買1個權重，2351買3個權重，那經過計算後花的錢分別是0.25、0.75。
## 買股票的賣出條件
- 今日最高價大於`close high price`
- 今日最低價小於`close low price`
- 價格都介於`close high price`和`close low price`間，但股票已經買進`life`天

## 放空股票的買入條件
- 今日最高價小於`close high price`
- 今日最低價大於`close low price`
- 價格都介於`close high price`和`close low price`間，但股票已經放空`life`天

## 未操作懲罰
如果當日沒有任何股票操作(沒有 decision.json 或是檔案內容是空的)，等同於投資大盤(TAIEX, 0050)，而根據過往經驗通常跟著大盤投資都會賠錢。
  
投資大盤的decision file如下:

In [ ]:
[
  {
    "code": "0050",
    "life": 1,
    "type": "buy",
    "weight": 1,
    "open_price": 'that days opening price',
    "close_high_price": 'that days highest price + 1',
    "close_low_price":' that days lowest price - 1'
  }
]

## 股票繳交要求

每個人的帳號底下都有一個stock資料夾，其中的結構為:  
- `~/stock/bin`
- `~/stock/commit`
- `~/stock/output` 
  
以下流程為一個範例，詳細的內容及名稱可任意更改:  
  
**1. 產生預測資料**  
寫一個如`generate_predict_data.py`的程式來準備預測資料，因為每天的日期不同，這隻程式通常會需要當天日期作為argv imput。 
      
**2. 產生Decision File**  
寫一個如`make_decision.py`的程式來產生Decision File，根據預測的結果以及額外條件判斷產生出當天所要購買的股票清單，格式如上述的JSON形式。輸出檔案必須放置在`~/stock/commit`資料夾中，檔名必須由預測當日日期構成，例如:`2018-09-10_2018-09-10.json`就代表2018-09-10這天你所購買的股票。  
  
**3. run.sh**  
`~/stock/bin`資料夾中還必須有一個叫做`run.sh`的程式(檔名不能更改)，`run.sh`為shell script([語法請參考這裡](http://blog.jex.tw/blog/2013/06/08/shell-script/))，裡面必須包含整個預測股票的流程，例如: 先執行`generate_predict_data.py`，接著執行`make_decision.py`等等，當`run.sh`成功執行之後，`~/stock/commit/`下會產生出當天的Decision File。  
  
在執行`run.sh`時，會在後面加上兩個日期，以上述2018-09-10當天的預測為例，指令為`sh run.sh 2018-09-10 2018-09-10`，其中第一個日期和第二個日期在shell script中分別代表`$1`、`$2`。
  
以下為`run.sh`的範例，

In [ ]:
# if command is sh run.sh 2018-09-10 2018-09-10

# Generate the predicting data
./generate-predict-data.py $1 5 ../output/predict-data 

# Predict
# using existing trained model to predict
./predict.py ../output/predict-data ../output/predict-result

# Make decision
./make-decision.py ../output/predict-result ../output/predict-stock-list ../commit/$1_$2.json
# The output file will be ../commit/2018-09-10_2018-09-10.json

**4. 備註**  
- 在運作上助教會將每個人的`~/stock/bin/`資料夾複製並執行，所以原本存在於`~/stock/bin/`以外的檔案都會遺失，若有需要中途產生的檔案供之後`make_decision.py`使用，則須將中途檔案存入`~/stock/output/`中存放。  
- 通常來說訓練模型會預先做好，模型會放置在`~/stock/bin/`中直接使用，由於系統在執行`run.sh`時有最多一分鐘的上限，所以每天重新訓練模型通常會超過時限而導致產生Decision File失敗。  
- 在網頁上的TEST可以測試自己目前的`run.sh`能不能順利產生Decision File，可以用來檢測目前程式能不能在系統上運行。
- 在網頁上的Stock-sim可以做精進模型的測試，運行stock-sim會在自己的luffy中產生結果，可以評估某種策略的好壞。